In [9]:
import os 
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd # type: ignore

In [10]:
from utils import get_dataset, get_model_by_tag, get_prompt_template
from LLMAnnotator import LLMAnnotator

In [11]:
os.environ['OPENAI_API_KEY_CLARIN']=os.getenv("OPENAI_API_KEY_CLARIN")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

token = os.getenv("OPENAI_API_KEY_CLARIN")

In [12]:
model_tag = 'llama3'  
dataset_tag = 'yahoo'
prompt = 'cot_random_samples_cohere'
temp = 0.3
number_of_experimet = 1
selected_samples =  f'{model_tag}_random42_60'

In [13]:
path = f'results/cot_random_samples_cohere_temp0.3/llama3/yahoo_llama3_cot_random_samples_cohere_llama3_random42_60_temp0.3_exp{number_of_experimet}.csv'

In [6]:
model = get_model_by_tag(model_tag, token, temp)  
dataset_for_annotation = get_dataset(dataset_tag=dataset_tag)
prompt_txt = get_prompt_template(dataset_tag, prompt)
examples_for_prompt = pd.read_csv(f'./selected_samples/{dataset_tag}/{selected_samples}.csv')

output_path = f'./results/{prompt}_temp{temp}/{model_tag}/{dataset_tag}_{model_tag}_{prompt}_{selected_samples}_temp{temp}_exp{number_of_experimet}.csv'
os.makedirs(f'./results/{prompt}_temp{temp}/{model_tag}', exist_ok=True)


In [14]:
text_col = "text"
label_col = "label"
output_col = "output"

In [15]:
import pandas as pd
import re

def extract_bracketed_word(df, read_col, write_col):
    """
    Wyodrębnia pierwszy wyraz objęty nawiasami [] z kolumny `read_col`
    i zapisuje go do kolumny `write_col`.
    
    :param df: DataFrame wejściowy
    :param read_col: Nazwa kolumny z której odczytywany jest tekst
    :param write_col: Nazwa kolumny do której zapisywany będzie wynik
    :return: DataFrame z nową kolumną
    """
    def extract_last_bracketed(text):
        matches = re.findall(r'\[([^\[\]]+?)\]', str(text))
        return matches[-1] if matches else None
    
    df[write_col] = df[read_col].apply(extract_last_bracketed)
    return df


import pandas as pd

def validate_text_column(path, df_reference):
    # Wczytaj DataFrame z pliku CSV
    df = pd.read_csv(path)
    
    # Pobierz unikalne wartości z kolumny 'text' z df_reference
    reference_texts = set(df_reference['text'].unique())
    
    # Dodaj kolumnę 'is_valid'
    df['was_in_selected_samples'] = df['text'].isin(reference_texts)
    df = extract_bracketed_word(df, 'output', 'extracted_label')
    df.to_csv(path, index=False)
    
    return df

In [16]:
def reannotate_errors(output_path, model, dataset, examples_for_prompt, prompt_txt, text_col="text", label_col="label", output_col="output"):
    # Wczytaj pierwotny DataFrame
    df_full = pd.read_csv(output_path)
    
    # Sprawdź, czy kolumna 'error' istnieje
    if 'error' not in df_full.columns:
        print("Brak kolumny 'error' – nie znaleziono błędów do ponownej adnotacji.")
        return df_full
    
    # Wybierz tylko wiersze z błędami
    df_errors = df_full[df_full['error'].notnull()].copy()
    
    if df_errors.empty:
        print("Brak wierszy z błędami do przetworzenia.")
        return df_full

    print(f"Znaleziono {len(df_errors)} wierszy z błędami. Przetwarzanie ponowne...")

    # Tworzymy adnotator tylko dla błędnych przykładów
    annotator = LLMAnnotator(
        model=model,
        dataset=dataset,
        examples_for_prompt=examples_for_prompt,
        prompt_template=prompt_txt,
        column_text=text_col,
        column_label=label_col,
        column_output=output_col
    )
    
    # Ponowna adnotacja tylko błędnych danych
    df_errors_fixed = annotator.get_results()
    
    # Upewniamy się, że indeksy pasują
    df_full.update(df_errors_fixed)

    # Zapisz zaktualizowany DataFrame
    df_full.to_csv(output_path, index=False)
    
    print("Aktualizacja zakończona.")
    return df_full


In [ ]:
df_full = reannotate_errors(path, model, dataset_for_annotation, examples_for_prompt, prompt_txt, text_col=text_col, label_col=label_col, output_col=output_col)

Znaleziono 77 wierszy z błędami. Przetwarzanie ponowne...


/home/julita/Desktop/anote-skrpyty 19.04/LLMAnnotator.py:77: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(llm=self.model, prompt=self.prompt)


Nr: 0, próba 1/30 – błąd: Request failed with status 429: {"detail":"The model is currently overloaded. Please try again later."}
Nr: 0, próba 2/30 – błąd: Request failed with status 429: {"detail":"The model is currently overloaded. Please try again later."}
Nr: 0, próba 3/30 – błąd: Request failed with status 429: {"detail":"The model is currently overloaded. Please try again later."}
Nr: 0, próba 4/30 – błąd: Request failed with status 429: {"detail":"The model is currently overloaded. Please try again later."}
Nr: 0, próba 5/30 – błąd: Request failed with status 429: {"detail":"The model is currently overloaded. Please try again later."}
Nr: 0, próba 6/30 – błąd: Request failed with status 429: {"detail":"The model is currently overloaded. Please try again later."}


Exception ignored in: <Finalize object, dead>
Traceback (most recent call last):
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/multiprocessing/util.py", line 227, in __call__
    res = self._callback(*self._args, **self._kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/multiprocessing/managers.py", line 675, in _finalize_manager
    process.join(timeout=shutdown_timeout)
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/julita/anaconda3/envs/anote/lib/python3.12/multiprocessing/connection.py", line 1136, in wait
    ready = selector.select(timeout)
            ^^^^^^^^^^^^^^^^^^^^^

Nr: 0, próba 7/30 – błąd: Request failed with status 429: {"detail":"The model is currently overloaded. Please try again later."}
Nr: 0, próba 8/30 – błąd: Request failed with status 429: {"detail":"The model is currently overloaded. Please try again later."}
Nr: 0, próba 9/30 – błąd: Request failed with status 429: {"detail":"The model is currently overloaded. Please try again later."}
Nr: 0, próba 10/30 – błąd: Request failed with status 429: {"detail":"The model is currently overloaded. Please try again later."}
Nr: 0, próba 11/30 – błąd: Request failed with status 429: {"detail":"The model is currently overloaded. Please try again later."}
Nr: 0, próba 12/30 – błąd: Request failed with status 429: {"detail":"The model is currently overloaded. Please try again later."}
Nr: 0, próba 13/30 – błąd: Request failed with status 429: {"detail":"The model is currently overloaded. Please try again later."}
Nr: 0, próba 14/30 – błąd: Request failed with status 429: {"detail":"The model is cur